In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
# from datasets import 

In [ ]:
model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer =  AutoTokenizer.from_pretrained(model_name)

In [ ]:
model.requires_grad_(False)